In [1]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import jieba as jb
import jieba.analyse
import codecs
import gensim
from gensim import corpora,models,similarities

## 1.Import Data

In [2]:
rows=pd.read_excel('data/Proj_Subj_Nm.xlsx',header=0,encoding='utf-8',dtype=str)

In [3]:
Proj_Nm = rows.Proj_Nm
Proj_list = Proj_Nm.tolist()

## 2.Clear Punctuation

In [4]:
import re

term_list = []

for i in Proj_list:
    Trm_Prj_Nm = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])"," ",i) #去除符號
    Prj_Nm = re.sub(r'\d+', ' NUM ',Trm_Prj_Nm)
    L_Prj_Nm = Prj_Nm.lower()
    term_list.append(L_Prj_Nm)

In [5]:
type(term_list)

list

In [80]:
name=['Trimmed_Proj_Nm']

In [83]:
Trimmed_Proj_Nm = pd.DataFrame(columns=name,data=term_list)

In [85]:
#print(term_list,file=open('Trimmed_Proj_Nm.txt','w'))
export_csv = Trimmed_Proj_Nm.to_csv (r'C:\Users\F78001\Desktop\term_list.csv', index = None, header=True)

## 3.Import Stop Words

In [5]:
#停止詞匯入一個數據組
stopwords = [line.strip() for line in codecs.open('data/stopwords.txt', 'r', 'utf-8').readlines()]
#载入停用词
jieba.analyse.set_stop_words('data/stopwords.txt')

In [6]:
#载入自訂辭庫
jieba.load_userdict('data/SpecialWords.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\F78001\AppData\Local\Temp\jieba.cache
Loading model cost 0.879 seconds.
Prefix dict has been built succesfully.


## 4.Parsing & Tokenize Terms

In [25]:
segments = []

for index in term_list:
    content = index #針對第2欄抽詞斷字 #TextRank 关键词抽取，只获取固定词性
    words = jieba.cut(content)
    splitedStr = ''
    for word in words:
        #停用词判断，如果当前的关键词不在停用词库中才进行记录
        if word not in stopwords:
            # 记录全局分词
            segments.append(word)
            splitedStr += word + ' '

In [27]:
for index in term_list:
    content = index
    #TextRank 关键词抽取，只获取固定词性
    words = jieba.analyse.textrank(content, topK=20,withWeight=False) #不篩選詞性
    splitedStr = ''
    for word in words:
        # 记录全局分词
        segments.append(word)
        splitedStr += word + ' '

In [32]:
segments_str = ",".join(segments) #轉為STR

In [34]:
all_doc_list = []
for doc in segments:
    doc_list = [word for word in jieba.cut(doc)]
    all_doc_list.append(doc_list)

In [35]:
dictionary = corpora.Dictionary(all_doc_list)

In [36]:
dictionary.keys()

[1113,
 818,
 263,
 105,
 806,
 704,
 845,
 274,
 164,
 605,
 331,
 1157,
 718,
 1126,
 1054,
 183,
 829,
 464,
 169,
 1191,
 980,
 1136,
 248,
 727,
 656,
 863,
 1013,
 1192,
 176,
 630,
 496,
 1,
 764,
 815,
 1059,
 78,
 1001,
 701,
 1000,
 590,
 1209,
 173,
 86,
 91,
 432,
 708,
 759,
 877,
 831,
 56,
 822,
 266,
 138,
 684,
 75,
 14,
 79,
 0,
 298,
 324,
 1080,
 301,
 189,
 901,
 210,
 341,
 369,
 790,
 16,
 221,
 571,
 855,
 687,
 846,
 669,
 435,
 914,
 506,
 568,
 835,
 21,
 770,
 599,
 101,
 749,
 802,
 755,
 191,
 800,
 1185,
 1071,
 1155,
 1063,
 949,
 1107,
 80,
 96,
 866,
 1073,
 843,
 333,
 594,
 363,
 480,
 467,
 958,
 307,
 894,
 372,
 386,
 518,
 269,
 543,
 665,
 761,
 945,
 422,
 1147,
 1021,
 400,
 457,
 64,
 154,
 198,
 707,
 194,
 396,
 186,
 489,
 353,
 1006,
 1051,
 112,
 613,
 1056,
 534,
 289,
 898,
 1029,
 436,
 603,
 260,
 102,
 395,
 290,
 60,
 1187,
 626,
 332,
 719,
 366,
 702,
 677,
 162,
 402,
 859,
 1035,
 72,
 1186,
 403,
 528,
 628,
 149,
 167,
 1139,

In [37]:
dictionary.token2id

{'檔案': 1113,
 '弾': 818,
 'draw': 263,
 '國泰世華': 105,
 '押': 806,
 '搭機': 704,
 '溫馨': 845,
 '美國運通': 274,
 '日增': 164,
 '享受': 605,
 '越南': 331,
 '高卡籍': 1157,
 'ff': 718,
 '無卡': 1126,
 'chinees': 1054,
 '簡': 183,
 '非美國': 829,
 'mit': 464,
 'extra': 169,
 '來襲': 1191,
 'amsterdam': 980,
 '好消息': 1136,
 'susanyuan': 248,
 'online': 727,
 '上海': 656,
 'shining': 863,
 'days': 1013,
 '報小': 1192,
 '賞星樂慕': 176,
 '時間': 630,
 'b': 496,
 'num': 1,
 'gw': 764,
 'atm': 815,
 'eletter': 1059,
 'campaign': 78,
 '楽': 1001,
 '近': 701,
 'business': 1000,
 '海莉': 590,
 '好': 1209,
 'abc': 173,
 '限時': 86,
 'de': 91,
 'discount': 432,
 '援': 708,
 '促銷代碼': 759,
 '見': 877,
 '天鵝湖': 831,
 'fare': 56,
 'flight': 822,
 'offer': 266,
 'astindo': 138,
 '京': 684,
 '西亞': 75,
 '簡體': 14,
 'lon': 79,
 ' ': 0,
 '至亞洲': 298,
 '三生': 324,
 'noch': 1080,
 'sale': 301,
 'sales': 189,
 'norden': 901,
 'ho': 210,
 '群組': 341,
 'tc': 369,
 '臺北': 790,
 '樂購': 16,
 'aud': 221,
 '正確': 571,
 'period': 855,
 '二月': 687,
 '五月': 846,
 '卡友案': 669,
 '獲

In [38]:
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]

## TF-IDF(Term Clustering)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans  
#from data_utils import *
import matplotlib.pyplot as plt

In [46]:
#1.加載語料
corpus = []
with open("data/Proj_Nm.txt", "r", encoding="utf-8") as f:
    for line in f:
        # 去掉标点符号
        corpus.append(segment_jieba(remove_punc(line.strip())))

In [47]:
#2.計算tf-idf設為權重
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(segments_str))

In [48]:
#3.獲取詞袋模型中的所有詞語特徵，如果特徵數量非常多的情況下可以按照權重降維
word = vectorizer.get_feature_names()
print("word feature length: {}".format(len(word)))

word feature length: 1067


In [54]:
#4.匯出權重，到這邊就實現了將文字向量化的過程，矩陣中的每一行就是一個文檔的向量表示
tfidf_weight = tfidf.toarray()

In [42]:
segments1 = []

for index, row in rep_rows.iterrows():
    content = row[1] #針對第2欄抽詞斷字
    #TextRank 关键词抽取，只获取固定词性
    words = jieba.cut(content)
    splitedStr = ''
    for word in words:
        #停用词判断，如果当前的关键词不在停用词库中才进行记录
        if word not in stopwords:
            # 记录全局分词
            segments1.append({'word':word, 'count':1})
            splitedStr += word + ' '

In [43]:
for index, row in rep_rows.iterrows():
    content = row[1]
    #TextRank 关键词抽取，只获取固定词性
    words = jieba.analyse.textrank(content, topK=20,withWeight=False) #不篩選詞性
    splitedStr = ''
    for word in words:
        # 记录全局分词
        segments1.append({'word':word, 'count':1})
        splitedStr += word + ' '

In [45]:
# 将结果数组转为df序列
dfSg = pd.DataFrame(segments1)

# 词频统计
dfWord = dfSg.groupby('word')['count'].sum()

In [46]:
#导出csv
dfWord.to_csv('Proj_keywords.csv',encoding='utf-8')

## Extract Part of Speech

In [75]:
for index, row in rep_rows.iterrows():
    content = row[2] #針對第3欄抽詞斷字
    #TextRank 关键词抽取，只获取固定词性
    words = jieba.analyse.textrank(content, topK=20,withWeight=False,allowPOS=('ns', 'n', 'vn', 'v'))
    splitedStr = ''
    for word in words:
        # 记录全局分词
        segments1.append({'word':word, 'count':1})
        splitedStr += word + ' '